In [3]:
%%time
# 이미지 스플릿 하기
# https://inhovation97.tistory.com/36

import os
import glob
import math
import shutil
import random

# 원본 이미지 폴더속 이미지 개수
doc_img = sorted(glob.glob('D:/coding/data_set/lomin/train_copy/train/'+'/*'))
print('doc_img 폴더 이미지 개수 : {}'.format(len(doc_img)))

# 나눌 이미지 수
doc_split = round(len(doc_img)*0.05)
print('doc_split : split 이미지 수 : {}'.format(doc_split))

# 이미지 나누기 정의
def split(img_list, test_count, train_path, test_path):

    test_files=[]
    for i in random.sample( img_list, test_count ):
        test_files.append(i)
    
    # 차집합으로 train/test 리스트 생성하기
    train_files = [x for x in img_list if x not in test_files]

    for k in train_files:
        shutil.copy(k, train_path)

    for c in test_files:
        shutil.copy(c, test_path)

    print('train 폴더 이미지 개수 : {}\n test 폴더 이미지 개수 : {}'.format(len(glob.glob(train_path+'/*')), len(glob.glob(test_path+'/*'))))

doc_img 폴더 이미지 개수 : 0
doc_split : split 이미지 수 : 0
Wall time: 2 ms


In [3]:
# 이미지 나누기 (각 지정 폴더로 들어감)
tr_path = 'D:/coding/data_set/lomin/temp_test/train'
te_path = 'D:/coding/data_set/lomin/temp_test/test'

split(doc_img, doc_split, tr_path, te_path)

train 폴더 이미지 개수 : 9080
 test 폴더 이미지 개수 : 478


---

In [4]:
# 이미지 여백만들고 회전시키기 (원본)

import os
import glob
import cv2

test_path = 'D:/coding/data_set/lomin/temp_test/test' # test path

img_list = sorted(glob.glob(test_path + '/*'))

for i in img_list:
    # print(i.split('\\')[1].split('.')[0])
    src = cv2.imread(i, cv2.IMREAD_COLOR)
    height, width, channel = src.shape
    my_angle = 0

    ##                                  중심점(center), 각도(angle), 비율(scale)
    matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
    # 비율 : 가로길이 / 대각선

    dst = cv2.warpAffine(src, matrix, (width, height))

    # 변경 이미지 저장 (파일명_각도)
    cv2.imwrite('D:/coding/data_set/lomin/temp_test/test_after/' + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)



In [93]:
## 설정 회전각 1개의 각에 의해 저장
import os
import glob
import cv2

input_path = 'D:/coding/data_set/lomin/train_copy/train' 
output_path = 'D:/coding/data_set/lomin/train_copy/train_a/'

img_list = sorted(glob.glob(input_path + '/*'))

# 설정 각도에 따라 저장.
def lotation_img(my_angle):
    for i in img_list:
        # print(i.split('\\')[1].split('.')[0])
        src = cv2.imread(i, cv2.IMREAD_COLOR)
        height, width, channel = src.shape
        
        ##                                  중심점(center), 각도(angle), 비율(scale)
        matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
        # 비율 : 가로길이 / 대각선

        dst = cv2.warpAffine(src, matrix, (width, height))

        # 변경 이미지 저장 (파일명_각도)
        # cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '_' + str(my_angle) + '.png', dst)
        
        cv2.imwrite(output_path + i.split('\\')[1].split('.')[0] + '.png', dst)

In [ ]:
lotation_img(0)

---

In [9]:
# Annotation 새로 만들기
#https://junyoung-jamong.github.io/machine/learning/2019/01/23/%EB%B0%94%EC%9A%B4%EB%94%A9%EB%B0%95%EC%8A%A4%EB%A5%BC-%ED%8F%AC%ED%95%A8%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%A6%9D%ED%8F%AD%EC%8B%9C%ED%82%A4%EA%B8%B0-with-imgaug.html

# 바운딩박스 복붙 (Annotation정보 읽기)
import os
import glob
import cv2
import xml.etree.ElementTree as ET

# test path
# test_path = 'D:/coding/data_set/lomin/train_copy/train' 

# 이미지 주소
test_img_path = 'D:/coding/data_set/lomin/train_copy/val/' 
# test_img_path = 'D:/coding/data_set/lomin/train_copy/train/' 

# 폴더 내 이미지 리스트
img_list = sorted(glob.glob(test_img_path + '/*'))

# Annotation 반복 만들기
for i in img_list:
	File = open(test_img_path + i.split('\\')[1].split('.')[0] + ".xml", "w")
	print("""<annotation>
	<folder>test_after</folder>
	<filename>{}.png</filename>
	<path>D:\coding\data_set\lomin\\train_copy\\train\\{}.png</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1654</width>
		<height>2339</height>
		<depth>1</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>0</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>355</xmin>
			<ymin>503</ymin>
			<xmax>1299</xmax>
			<ymax>1836</ymax>
		</bndbox>
	</object>
</annotation>
	""".format(i.split('\\')[1].split('.')[0], i.split('\\')[1].split('.')[0]), file=File)

	File.close()

---

In [14]:
%%time
## 이미지와 xml 파일을 랜덤으로 골라 각도별 폴더에 복사 붙여넣기 (memory out 방지)
## 주의!! annotation 은 회전된 데이터가 아님
## mtrixdml 각도를 0으로 고정하면, 배경만 추가한 데이터 생성
import glob
import numpy as np
import cv2
import shutil

dir = 'D:/coding/data_set/lomin/train_copy/train/'

# # 새 파일 넣을 경로 및 폴더 만들기 (train)
# dir_after = 'D:/coding/data_set/lomin/train_copy/train_angle/'
# os.makedirs(dir_after)

# 새 파일 넣을 경로 및 폴더 만들기 (val)
dir_after = 'D:/coding/data_set/lomin/train_copy/val_angle/'
os.makedirs(dir_after)

# dir 에 png 파일 선택 (내림차순)
dir_png = sorted(glob.glob(dir+'/*.png'))

## 각 폴더마다 몇개의 샘플을 만들 것인가?
num_img = 5

# 원하는 각도 만큼 폴더 생성
for i in range(0,360):
    
    # dir의 png 파일 중 랜덤으로 n개 선택 (비중복)
    png_random = sorted(np.random.choice(dir_png, num_img, replace=False))
    
    my_angle = i

    # 각도별 폴더 생성
    os.makedirs(dir_after + str(my_angle)) 
    
    for idx in png_random:
        src = cv2.imread(idx, cv2.IMREAD_COLOR)
        height, width, channel = src.shape
        ##                                  중심점(center), 각도(angle), 비율(scale)
        matrix = cv2.getRotationMatrix2D((width/2, height/2), 0, 0.57)
        # matrix = cv2.getRotationMatrix2D((width/2, height/2), my_angle, 0.57)
        dst = cv2.warpAffine(src, matrix, (width, height))

        # 이미지 파일 저장 경로
        png_random = idx.split('\\')[1].split('.')[0] + '.png'
        cv2.imwrite(dir_after + str(my_angle) + '/' + png_random, dst)
            
        # 랜덤 이미지와 같은 xml 파일을 복사
        xml_random = idx.split('\\')[0]+'/'+idx.split('\\')[-1].split('.')[0]+'.xml'
        shutil.copy(xml_random, dir_after + str(my_angle) + '/')


Wall time: 2min 54s


---

In [15]:
## Annotation 읽기

# https://junyoung-jamong.github.io/machine/learning/2019/01/23/%EB%B0%94%EC%9A%B4%EB%94%A9%EB%B0%95%EC%8A%A4%EB%A5%BC-%ED%8F%AC%ED%95%A8%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%A6%9D%ED%8F%AD%EC%8B%9C%ED%82%A4%EA%B8%B0-with-imgaug.html

import xml.etree.ElementTree as ET

def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [16]:
## 이미지 데이터 읽기

from os import listdir
import cv2
import numpy as np

def read_train_dataset(dir):
    images = []
    annotations = []

    for file in listdir(dir):
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(dir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    # images = np.array(images)

    return images, annotations

In [22]:
%%time
####### 폴더 추가해서 복사하기 ####################

## 테두리가 만들어진 이미지 폴더들을 순화하며 폴더 명에 따라 회전 시키기

## 순환이 문자로(j) 인식되기 때문에 0, 1, 10, 100, 101, 102, 103... 109, 11, 110.. 순으로 작업 수행

import imgaug as ia
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

ia.seed(1)

# 새 파일 넣을 경로 및 폴더 만들기 (train)
dir_after_ro = 'D:/coding/data_set/lomin/train_copy/train_angle_ro/'
os.makedirs(dir_after_ro)

# # 새 파일 넣을 경로 및 폴더 만들기 (val)
# dir_after_ro = 'D:/coding/data_set/lomin/train_copy/val_angle_ro/'
# os.makedirs(dir_after_ro)


# 파일 읽을 경로
dir = 'D:/coding/data_set/lomin/train_copy/train_angle/' # train 테스트셋
# dir = 'D:/coding/data_set/lomin/train_copy/val_angle/' # val 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    # os.makedirs(dir_after_ro + '/dgr_' + j) 

    # 각 폴더에 이미지 저장
    # dir_dgr = dir_after_ro + '/dgr_' + j + '/'

    # 한 폴더에 이미지 모두 저장
    dir_dgr = dir_after_ro
    

    images, annotations = read_train_dataset(dir_d)

    for idx in range(len(images)):
        image = images[idx]
        boxes = annotations[idx][0]

        ia_bounding_boxes = []
        for box in boxes:
            ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))

        bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

        seq = iaa.Sequential([
            # iaa.Multiply((1.2, 1.5)),
            iaa.Affine(
                # translate_px={"x": 40, "y": 60},
                # scale=(0.5, 0.7),
                rotate = int(j)
            )
        ])

        seq_det = seq.to_deterministic()

        image_aug = seq_det.augment_images([image])[0]
        bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

        # new_image_file = dir + 'after_' + annotations[idx][2]
        # new_image_file = dir_d + annotations[idx][2]
        new_image_file = dir_dgr + annotations[idx][2]
        cv2.imwrite(new_image_file, image_aug)

        h, w = np.shape(image_aug)[0:2]
        voc_writer = Writer(new_image_file, w, h)

        for i in range(len(bbs_aug.bounding_boxes)):
            bb_box = bbs_aug.bounding_boxes[i]
            # name (label) , x1, x2, y1, y2 값 바꿈
            voc_writer.addObject(j, int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

        # voc_writer.save(dir + 'after_' + annotations[idx][1])
        # voc_writer.save(dir_d + annotations[idx][1])
        voc_writer.save(dir_dgr + annotations[idx][1])


NotADirectoryError: [WinError 267] 디렉터리 이름이 올바르지 않습니다: 'D:/coding/data_set/lomin/train_copy/train_angle/desktop.ini/'

In [74]:
%%time
####### 폴더 추가해서 복사하기 ####################

## 테두리가 만들어진 이미지 폴더들을 순화하며 폴더 명에 따라 회전 시키기

## 순환이 문자로(j) 인식되기 때문에 0, 1, 10, 100, 101, 102, 103... 109, 11, 110.. 순으로 작업 수행

import imgaug as ia
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

ia.seed(1)

# dir = 'D:/coding/data_set/lomin/train_copy/train_angle/' # train 테스트셋
# dir = 'D:/coding/data_set/lomin/train_copy/val_angle/' # val 테스트셋

dir = 'D:/coding/data_set/lomin/temp/val_angle/' # val 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    # os.makedirs(dir_d + '/dgr_' + j) 
    dir_2 = dir_dgr = dir_d + '/dgr_' + j + '/'
    
    f = dir_2 + '*'
    print(f)

D:/coding/data_set/lomin/temp/val_angle/117//dgr_117/*
D:/coding/data_set/lomin/temp/val_angle/2//dgr_2/*
D:/coding/data_set/lomin/temp/val_angle/44//dgr_44/*
D:/coding/data_set/lomin/temp/val_angle/55//dgr_55/*
Wall time: 974 µs


---

In [ ]:
%%time
## 이미 0으로 적용된 name 을 각도 별로 다시 지정 (1회용)

import imgaug as ia
import glob
import os
from imgaug import augmenters as iaa
# from files import *
from pascal_voc_writer import Writer

import xml.etree.ElementTree as ET

ia.seed(1)

# dir = 'D:/coding/data_set/lomin/train_copy/train_angle/' # train 테스트셋
# dir = 'D:/coding/data_set/lomin/train_copy/val_angle/' # val 테스트셋

dir = 'D:/coding/data_set/lomin/temp/val_angle/' # val 테스트셋

# 모든 폴더 순환
for j in sorted(os.listdir(dir)):
    dir_d = dir + j + '/' 

    # os.makedirs(dir_d + '/dgr_' + j) 
    dir_2 = dir_dgr = dir_d + 'dgr_' + j #+ '/'
    
    # ---
    # f = dir_2
    # label = f.split('/')[-2].split('_')[1]
    
    # print(label)
    # # print(os.listdir(f))
    # dir_png = sorted(glob.glob(dir_2+'/*.xml'))
    # print(dir_png)
    
    # # for i in os.listdir(f):
    # ---
    

    ## 폴더안에 있는 모든 xml 파일 이름 가져오기
    file_list = os.listdir(dir_2)
    xml_list = []
    
    for file in file_list:
        if '.xml' in file:
            xml_list.append(file)

    ## 모든 .xml파일에 대해 수정
    for xml_file in xml_list:
        target_path = dir_2 + "/" + xml_file
        targetXML = open(target_path, 'rt', encoding='UTF8')

        tree = ET.parse(targetXML)

        root = tree.getroot()
        
        ##수정할 부분
        target_tag = root.find("object/name")  # boject 하위 name 

        original = target_tag.text     #원본 String
        modified = original.replace(r"0",j)  # 0을 j 로 바꿈
        # modified = modified.replace("/", "\\")
        target_tag.text = modified  #수정
        
        tree.write(target_path)
        
## https://skkim1080.tistory.com/entry/Python%EC%9C%BC%EB%A1%9C-xml-%ED%8C%8C%EC%9D%BC-%EB%82%B4%EC%9A%A9-%EC%88%98%EC%A0%95%ED%95%98%EA%B8%B0



---

In [121]:
from PIL import Image
aaa = Image.open(idx + '000876.png')
imgArray = np.asarray(aaa)
print(imgArray.shape)

(2339, 1654, 3)
